In [72]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [6]:
data = pd.read_csv('P:/python/Fraud_check.csv')
data

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [7]:
data = data.rename({'Marital.Status':'martial_status', 'Taxable.Income':'tax_income', 'City.Population':'city_pop', 'Work.Experience':'work_exper'}, axis = 1)
data.head()

,Undergrad,martial_status,tax_income,city_pop,work_exper,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [10]:
data1 = data.copy()

In [11]:
data1['tax_income'] = pd.cut(data1['tax_income'], bins = [10002,30000,99619], labels = ['risky', 'good'])
data1.head()

,Undergrad,martial_status,tax_income,city_pop,work_exper,Urban
0,NO,Single,good,50047,10,YES
1,YES,Divorced,good,134075,18,YES
2,NO,Married,good,160205,30,YES
3,YES,Single,good,193264,15,YES
4,NO,Married,good,27533,28,NO


In [14]:
le = LabelEncoder()
data1['Undergrad'] = le.fit_transform(data1['Undergrad'])
data1['martial_status'] = le.fit_transform(data1['martial_status'])
data1['Urban'] = le.fit_transform(data1['Urban'])
data1['tax_income'] = le.fit_transform(data1['tax_income'])
data1.head()

,Undergrad,martial_status,tax_income,city_pop,work_exper,Urban
0,0,2,0,50047,10,1
1,1,0,0,134075,18,1
2,0,1,0,160205,30,1
3,1,2,0,193264,15,1
4,0,1,0,27533,28,0


In [25]:
x = data1.iloc[:,[0,1,3,4,5]]
y = data1.iloc[:,2]
print(x.head())
y.head()

   Undergrad  martial_status  city_pop  work_exper  Urban
0          0               2     50047          10      1
1          1               0    134075          18      1
2          0               1    160205          30      1
3          1               2    193264          15      1
4          0               1     27533          28      0


0    0
1    0
2    0
3    0
4    0
Name: tax_income, dtype: int64

In [62]:
x_train, x_test,y_train,y_test = train_test_split(x,y, test_size=0.25,random_state=40)
print(x_train.shape, x_test.shape,y_train.shape,y_test.shape)

(450, 5) (150, 5) (450,) (150,)


### Random Forest classification

In [95]:
num_trees = 100
max_features = 'auto'
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(x_train,y_train)

RandomForestClassifier()

In [96]:
preds = model.predict(x_test)
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [97]:
# using train test split
np.mean(preds==y_test)

0.74

In [111]:
# using kfold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10)
results_rf = cross_val_score(model, x, y, cv=kfold)
print(results_rf.mean())

0.735


### Bagging Classification

In [113]:
from sklearn.ensemble import BaggingClassifier
kfold = KFold(n_splits=10)
cart = DecisionTreeClassifier()
num_trees = 100
model_b = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=42)
results_b = cross_val_score(model_b, x_train, y_train, cv=kfold)
print(results_b.mean())

0.7333333333333333


In [110]:
results_b

array([0.8       , 0.64444444, 0.57777778, 0.73333333, 0.84444444,
       0.77777778, 0.77777778, 0.66666667, 0.75555556, 0.75555556])

### Adaboost classification

In [114]:
from sklearn.ensemble import AdaBoostClassifier
kfold = KFold(n_splits=10)
model_a = AdaBoostClassifier(n_estimators=10, random_state=42)
results_a = cross_val_score(model_a, x_train, y_train, cv=kfold)
print(results_a.mean())

0.7977777777777778


Firstly i have load a dataset and i have observed that the names of the columns are not familiar with python and i have done renaming. Then as in the problem statement i have divided taxable income lessthan 30000 as risky and remaining as good by the help of pandas cut method. After that with the labelencoder method i have done labelencoding for the necessary columns. After that i have divided the data into input and output by naming as x and y. Then by using trian_test_split ihave divided data into xtrian, ytrian, xtest, ytest by giving test = 25% and random state = 40. 
###### Random forest
Now i have build model with random forest classifier by giving no.of trees and number of features(columns) i have set features as auto. And i have fit x_trian and y_trian on model. Then i have done predictions on model by giving xtest then i check model accuarcy is 74% and by using kfold and cross val score i got accuarcy 73%
###### Bagging and boosting
And i have a model using bagging and boosting and i have got accuracy for bagging 73% and for boosting 79%